##### Scripts for:

[x]. Analyzing an audio file to extract/clone a voice,

[]. Generating new audio files from text in that cloned voice.

##### 1. Coqui TTS has voice cloning support where you can provide reference audio files and generate speech with that voice:

In [ ]:
import argparse
import os
import re
import sys
from pathlib import Path

import torch
from TTS.api import TTS

d:\Dhamma\AudioFileTool\audiogen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


In [4]:

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
# initialize TTS model
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2")

# clone from reference audio
reference_files = ["reference_voice.mp3"]
tts.tts_to_file(
    text="This is a test of my cloned voice!",
    speaker_wav=reference_files,
    language="en",
    file_path="output_cloned.wav"
)